#Criar um script que faça:  
#Pegue as nfe em xml do email  
#Extrai os dados necessarios   
#Tratar dados   
#Remover materiais pesado E outras notas 
-TABUAS CABOQUINHO  
-CIMENTRO VOTORAN  
-Magalu   

#Aplicador formula de taxas  
#Inserir no template dos marketplaces
  

In [10]:
import os
import shutil
import xml.etree.ElementTree as ET
import pandas as pd
from imap_tools import MailBox, AND
from env import pwd, user
from datetime import date, datetime
import requests
from bs4 import BeautifulSoup
import json
import time


In [11]:
#Baixa os anexos XML dos emails

save_folder = "notas/nfes/"
os.makedirs(save_folder, exist_ok=True)

with MailBox("imap.gmail.com").login(user, pwd, initial_folder="INBOX", ) as mailbox:
    list_mail = mailbox.fetch(criteria=AND(date=date.today())

    )
    for email in list_mail:
        for anexo in email.attachments:
            if  anexo.filename.lower().endswith(".xml"):
                file_path = os.path.join(save_folder, anexo.filename)
                if not os.path.exists(file_path):   
                    with open(file_path, "wb") as f:
                        f.write(anexo.payload)

In [12]:
# Salva os XMLs em pastas por Emitente e extrai os produtos para um Excel

pasta_origem = "notas/nfes"
todos_produtos = []


def extrai_dados(caminho_arquivo):
    produtos = []
    tree = ET.parse(caminho_arquivo)
    root = tree.getroot()
    ns = {"ns": "http://www.portalfiscal.inf.br/nfe"}
    data_str = root.find(".//ns:ide/ns:dhEmi", ns).text
    data_emissao = datetime.fromisoformat(data_str)

    
    natOp = root.find(".//ns:ide/ns:natOp", ns)
    if natOp is not None and natOp.text.strip().upper() == 'BONIFICACAO, DOACAO OU BRINDE':
        return [] 
    
    for det in root.findall(".//ns:det", ns):   
        produto = {}
        temp_codigo = det.find("./ns:prod/ns:cProd", ns).text
        if not "-" in temp_codigo:
            produto ['Codigo Produto'] = temp_codigo
        produto ['Descrição'] = det.find("./ns:prod/ns:xProd", ns).text
        produto ['Valor_unitário'] = det.find("./ns:prod/ns:vUnCom", ns).text
        produto ['Código de Barras'] = det.find("./ns:prod/ns:cEAN", ns).text 
        produto ['Sku'] = produto['Código de Barras']
        if produto['Sku'] == 'SEM GTIN':
            produto['Sku'] = produto['Descrição']
        produto ['Fornecedor'] = root.find(".//ns:emit/ns:xNome", ns).text
        produto ['Data Emissão'] = data_emissao
        produtos.append(produto)
    return produtos

caminho = 'notas/35200213571969000164550010013235011005754650-nfe-env.xml'    
produtos = extrai_dados(caminho)
todos_produtos.extend(produtos) 

# Percorre todos os arquivos da pasta
for arquivo in os.listdir(pasta_origem):
    if arquivo.lower().endswith(".xml"):
        caminho_arquivo = os.path.join(pasta_origem, arquivo)

        try:
            # Abre e parseia o XML
            tree = ET.parse(caminho_arquivo)
            root = tree.getroot()

            # Define namespace
            ns = {"ns": "http://www.portalfiscal.inf.br/nfe"}

            # Busca o CNPJ do emitente
            cnpj_emitente = root.find(".//ns:emit/ns:CNPJ", ns)
            cnpj_emitente
            if cnpj_emitente is None:
                print(f"⚠️ CNPJ não encontrado em {arquivo}")
                continue

            cnpj_emitente = cnpj_emitente.text

            produtos = extrai_dados(caminho_arquivo)
            todos_produtos.extend(produtos)              


            emitente = root.find(".//ns:emit/ns:xNome", ns)
            pasta_destino = os.path.join("notas/")

            # Cria a pasta se não existir
            os.makedirs(pasta_destino, exist_ok=True)

            # Copia o arquivo para a pasta
            shutil.copy(caminho_arquivo, pasta_destino)

        except Exception as e:
            print(f" {arquivo}: {e}")



fornecedores_pesados = [
    'IND E COM DE TUBOS E CONEXOES FORT.COM',
    'VOTORANTIM CIMENTOS SA',
    'CABOQUINHO MATERIAIS PARA CONSTRUCAO'
]


produtos = pd.DataFrame(todos_produtos)
produtos = produtos[~produtos['Fornecedor'].isin(fornecedores_pesados)]
produtos = produtos.sort_values(by="Data Emissão", ascending=False)

produtos = produtos.drop_duplicates(subset='Codigo Produto', keep='first')



In [13]:
produtos['Fornecedor'].unique()



array(['MJLESTE DISTRIBUIDORA DE MATERIAIS PARA CONSTRUCAO LTDA',
       'CONSTRUDIGI DISTRIBUIDORA DE MATERIAIS PARA CONSTRUCAO LTDA',
       'CONSTRUJA DISTR. DE MATERIAIS P/ CONSTRU',
       'M.S.B. COMERCIO DE MATERIAIS PARA CONSTRUCAO',
       'Saint-Gobain do Brasil Produtos Ind ustriais e para Construc',
       'MJR CUNHA DISTRIBUIDORA DE MAT.CONSTR LTDA',
       'SOOPER HOLDING LTDA', 'SOPREMA LTDA',
       'DENVER IMPERMEABILIZANTES, INDUSTRIA E C',
       'CARREFOUR COMERCIO E INDUSTRIA LTDA',
       'KADESH EQUIPAMENTOS PROFISSIONAIS LTDA',
       'Global Center Com.Ferragens Ltda',
       'MJR CUNHA DISTRIBUIDORA DE MATERIAIS PARA CONSTRUCAO LTDA',
       'MAGAZINE LUIZA SA'], dtype=object)

In [14]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
}



In [15]:
for  index, produto in produtos.iterrows():
    fornecedor = produto['Fornecedor']
    codigo_produto = produto['Codigo Produto']
    descricao = produto['Descrição']
    try:
        if fornecedor == 'CONSTRUDIGI DISTRIBUIDORA DE MATERIAIS PARA CONSTRUCAO LTDA':   
            url = f'https://www.construdigi.com.br/produto/{codigo_produto}/{descricao}'
        elif fornecedor == 'M.S.B. COMERCIO DE MATERIAIS PARA CONSTRUCAO':
            url = f'https://msbitaqua.com.br/produto/{codigo_produto}/{descricao}'
        elif fornecedor == "CONSTRUJA DISTR. DE MATERIAIS P/ CONSTRU":
            url = f'https://www.construja.com.br/produto/{codigo_produto}/{descricao}'
        else:
            print('Indisponivel')
            continue
        
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        script = soup.find("script", type="application/json")
        data = json.loads(script.string) if script else {}

        extrai = data.get("props", {}).get("pageProps", {}).get("produto", {})
        url_img = data.get("props", {}).get("pageProps", {}).get("seo", {}).get("imageUrl", 'Não disponivel')
        marca = next((p.get("desc") for p in extrai.get("dimensoes", []) if p.get("label") == "MARCA"), "Não disponível")
        peso = extrai.get("pesoBruto", "Não disponível")
        if codigo_barras == 'SEM GTIN':
            codigo_barras = extrai.get("codBarra", "SEM GTIN")
        produtos.at[index, 'Código de Barras'] = codigo_barras
        produtos.at[index, 'Peso'] = peso
        produtos.at[index, "Marca"] = marca
        produtos.at[index, 'Url Imagem'] = url_img
        print(produto['Descrição'])
        
    except Exception as e:
        print(e)


produtos.to_excel('produtos.xlsx', index=False)


Indisponivel
Indisponivel
Indisponivel
Indisponivel
Indisponivel
Indisponivel
Indisponivel
Indisponivel
Indisponivel
Indisponivel
Indisponivel
Indisponivel
Indisponivel
Indisponivel
Indisponivel
name 'codigo_barras' is not defined
name 'codigo_barras' is not defined
name 'codigo_barras' is not defined
name 'codigo_barras' is not defined
name 'codigo_barras' is not defined
name 'codigo_barras' is not defined
name 'codigo_barras' is not defined
name 'codigo_barras' is not defined
name 'codigo_barras' is not defined
name 'codigo_barras' is not defined
name 'codigo_barras' is not defined
name 'codigo_barras' is not defined
name 'codigo_barras' is not defined
name 'codigo_barras' is not defined
name 'codigo_barras' is not defined
name 'codigo_barras' is not defined
name 'codigo_barras' is not defined
name 'codigo_barras' is not defined
name 'codigo_barras' is not defined
name 'codigo_barras' is not defined
name 'codigo_barras' is not defined
name 'codigo_barras' is not defined
name 'codigo_

KeyboardInterrupt: 

In [ ]:
import json
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright




async def run():

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, wait_until="networkidle")

        # pega o HTML renderizado
        html = await page.content()
        soup = BeautifulSoup(html, "html.parser")
        # tenta pegar o JSON do __NEXT_DATA__
        script = soup.find("script", id="__NEXT_DATA__")
        if script:
            data = json.loads(script.string)
            produto = data.get("props", {}).get("pageProps", {}).get("produto", {})
            url_img = data.get("props", {}).get("pageProps", {}).get("seo", {}).get("imageUrl", 'Não disponivel')
            print(url_img)
            print("Marca:", next((p.get("desc") for p in produto.get("dimensoes", []) if p.get("label") == "MARCA"), "não disponível"))
            print("Peso:", produto.get("pesoBruto", "não disponível"))
            print("Código de Barras:", produto.get("codBarra", "não disponível"))

        else:
            print("❌ Nenhum <script id=__NEXT_DATA__> encontrado")

        await browser.close()




ValueError: Excel does not support datetimes with timezones. Please ensure that datetimes are timezone unaware before writing to Excel.

In [17]:
import requests
from bs4 import BeautifulSoup


response = requests.get('https://msbitaqua.com.br/produto/2123/2123')
soup = BeautifulSoup(response.text, "html.parser")
script = soup.find("script", type="application/json")
data = json.loads(script.string)

produto = data['props']['pageProps']
produto

{'appProps': {'banners': [{'id': 13,
    'nome': 'Banner Serlonas',
    'imagem_desktop': 'https://cdn.msbitaqua.com.br/arquivos/publico/banner_web_SERLONAS.jpg?ts=1725884055024.6',
    'imagem_mobile': 'https://cdn.msbitaqua.com.br/arquivos/publico/banner_mobile_SERLONAS.jpg?ts=1725884072716.8',
    'data_inicio': '2024-09-09',
    'data_fim': '2080-09-09',
    'ordem': 1,
    'nova_aba': True,
    'popup': False,
    'href': 'https://msbitaqua.com.br/produtos?pagina=1&busca=serlonas',
    'tipo': 'LINK_EXTERNO',
    'filiais': ['1', '2'],
    'local': 'HOME',
    'sistemas': [],
    'link_integrado_id': None},
   {'id': 10,
    'nome': 'Banner Compel',
    'imagem_desktop': 'https://cdn.msbitaqua.com.br/arquivos/publico/banner_web_COMPEL-01.jpg?ts=1724239271707',
    'imagem_mobile': 'https://cdn.msbitaqua.com.br/arquivos/publico/banner_mobile_COMPEL-01__1_.jpg?ts=1724239271683',
    'data_inicio': '2024-08-21',
    'data_fim': '2080-08-21',
    'ordem': 2,
    'nova_aba': False,
   

In [ ]:
import json
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright

async def run():

    async with async_playwright() as p:
        url = 'https://msbitaqua.com.br/produto/2123/2123'
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, wait_until="networkidle")

        # pega o HTML renderizado
        html = await page.content()
        soup = BeautifulSoup(html, "html.parser")
        # tenta pegar o JSON do __NEXT_DATA__
        script = soup.find("script", id="__NEXT_DATA__")
        if script:
            data = json.loads(script.string)
            print(data.keys())
            produto = data.get("props", {}).get("pageProps", {}).get("produto", {})
            url_img = data.get("props", {}).get("pageProps", {}).get("seo", {}).get("imageUrl", 'Não disponivel')
            print(url_img)
            print("Marca:", next((p.get("desc") for p in produto.get("dimensoes", []) if p.get("label") == "MARCA"), "não disponível"))
            print("Peso:", produto.get("pesoBruto", "não disponível"))
            print("Código de Barras:", produto.get("codBarra", "não disponível"))
        else:
            print("❌ Nenhum <script id=__NEXT_DATA__> encontrado")

        await browser.close()

# 👉 Em Jupyter/IPython:


await run()

Codigo Produto
Descrição
Valor_unitário
Código de Barras
Sku
Fornecedor
Data Emissão


In [1]:
import os
import shutil
import xml.etree.ElementTree as ET
import pandas as pd
from imap_tools import MailBox, AND
from env import pwd, user
from datetime import date, datetime
import requests
from bs4 import BeautifulSoup
import json
import time


import json
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright

async def run():

    async with async_playwright() as p:
        url = 'https://www.construdigi.com.br/produto/10422/multilit-tubo-pvc-esg-75'
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, wait_until="networkidle")

        # pega o HTML renderizado
        html = await page.content()
        soup = BeautifulSoup(html, "html.parser")
        # tenta pegar o JSON do __NEXT_DATA__
        script = soup.find("script", id="__NEXT_DATA__")
        if script:
            data = json.loads(script.string)
            produto = data.get("props", {}).get("pageProps", {}).get("produto", {})
            url_img = data.get("props", {}).get("pageProps", {}).get("seo", {}).get("imageUrl", 'Não disponivel')
            print(url_img)
            print("Marca:", next((p.get("desc") for p in produto.get("dimensoes", []) if p.get("label") == "MARCA"), "não disponível"))
            print("Peso:", produto.get("pesoBruto", "não disponível"))
            print("Código de Barras:", produto.get("codBarra", "não disponível"))
        else:
            print("❌ Nenhum <script id=__NEXT_DATA__> encontrado")

        await browser.close()

# 👉 Em Jupyter/IPython:


await run()


https://cdn.construdigi.com.br/produtos/10422/10422.jpg
Marca: MULTILIT
Peso: 1.88600000
Código de Barras: 7898394632067


In [2]:
Dados = {'marca': 'MULTILIT', 'peso': '1.88600000', 'codigo_barras': '7898394632067', 'url_img': 'https://cdn.construdigi.com.br/produtos/10422/10422.jpg'}


In [8]:
produtos = [{'Codigo Produto': '10422', 'Descrição': 'TUBO PVC ESG 75', 'Valor_unitário': '65.9400000000', 'Código de Barras': '7898394632067', 'Sku': '7898394632067', 'Fornecedor': 'CONSTRUDIGI DISTRIBUIDORA DE MATERIAIS PARA CONSTRUCAO LTDA', 'Data Emissão': ('2025-09-11 04:14:00')}] 

In [10]:
produtos[0].get('Código de Barras')

'7898394632067'